# nnfabrik example usage

In this notebook we will go through one example usage of nnfabrik pipeline. We have already defined our three functions ([dataset function](../nnfabrik/datasets/toy_datasets.py), [model function](../nnfabrik/models/toy_models.py), [trainer function](../nnfabrik/training/toy_trainers.py)). 

Remember the goal is to train models with different hyperparameters (batchsize, number of layer, learning rate, etc.). Therefore, what we are left with is to fill up the corresponding Datajoint tables with different hyper-parameter values and train models for every possible combination of those hyper-parameters

## Connect to the database

In [1]:
import os
import datajoint as dj
dj.config['database.host'] = os.environ['DJ_HOST']
dj.config['database.user'] = os.environ['DJ_USERNAME']
dj.config['database.password'] = os.environ['DJ_PASSWORD']
dj.config['enable_python_native_blobs'] = True
dj.config['schema_name'] = "nnfabrik_example"

from nnfabrik.main import *
import nnfabrik as nnf

# nnf.config['repos'] = ['/notebooks/lib/nnfabrik']

Connecting mohammad@sinzlab.chlkmukhxp6i.eu-central-1.rds.amazonaws.com:3306


## Fabrikant table

Fabrikant tables keeps a record of the users that interact with a specific schema. It is simply an extra level of information to know who is accountable for the entries in the Dataset, Model, and Trainer tables. simply add your information as follows:

In [2]:
fabrikant_info = dict(fabrikant_name="Your Name", email="your@email.com", affiliation="thelab", dj_username="yourname")
Fabrikant().insert1(fabrikant_info)

In [3]:
Fabrikant()

fabrikant_name Name of the contributor that added this entry,email e-mail address,affiliation conributor's affiliation,dj_username DataJoint username
Your Name,your@email.com,thelab,yourname


## Dataset table

Here we need to specify **dataset function** and the arguments passed to the dataset function, **dataset config**. The dataset function is specified as a string. the structure of this string is important since under the hood nnfabrik performs a dynamic import by parsing this string. For example, if you can import the function as
``` python
from nnfabrik.datasets import toy_dataset_fn
```
then you should specify the dataset function as:
``` python
"nnfabrik.datasets.toy_dataset_fn"
```

**Note**: Since nnfabrik would need to import the function, the dataset function must be importable. <br>
Also **Note** that dataset config is a dictionary that contains all the arguments that are NOT required by nnfabrik.

In [4]:
# specify dataset function as string (the function must be importable) as well as the dataset config
dataset_fn = "nnfabrik.datasets.toy_dataset_fn"
dataset_config = dict(batch_size=64) # we specify all the inputs except the ones required by nnfabrik

Dataset().add_entry(dataset_fn=dataset_fn, dataset_config=dataset_config, 
                    dataset_fabrikant="Your Name", dataset_comment="A comment about the dataset!");

In [5]:
Dataset()

dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,dataset_config dataset configuration object,dataset_fabrikant Name of the contributor that added this entry,dataset_comment short description,dataset_ts UTZ timestamp at time of insertion
nnfabrik.datasets.toy_dataset_fn,9aee736870714f8b7c3cc084087ce886,=BLOB=,Your Name,A comment about the dataset!,2020-06-13 10:00:53


## Model table

Here we need to specify **dataset function** and the arguments passed to the dataset function, **dataset config**. Everything explained for the dataset function applied to model function as well.

In [6]:
# specify model function as string (the function must be importable) as well as the model config
model_fn = "nnfabrik.models.toy_model_fn"
model_config = dict(h_dim=5) # we specify all the inputs except the ones required by nnfabrik

Model().add_entry(model_fn=model_fn, model_config=model_config, 
                  model_fabrikant="Your Name", model_comment="A comment about the model!");

Let's also try `h_dim = 15`:

In [7]:
model_config = dict(h_dim=15) # we specify all the inputs except the ones required by nnfabrik
Model().add_entry(model_fn=model_fn, model_config=model_config, 
                  model_fabrikant="Your Name", model_comment="A comment about the model!");

In [8]:
Model()

model_fn name of the model function,model_hash hash of the model configuration,model_config model configuration to be passed into the function,model_fabrikant Name of the contributor that added this entry,model_comment short description,model_ts UTZ timestamp at time of insertion
nnfabrik.models.toy_model_fn,24922759b843076328c4f3b9df3f88d0,=BLOB=,Your Name,A comment about the model!,2020-06-13 10:00:54
nnfabrik.models.toy_model_fn,46b0c26ef504ff0eddf55b15550a4f98,=BLOB=,Your Name,A comment about the model!,2020-06-13 10:00:57


## Trainer table

Here we need to specify **trainer function** and the arguments passed to the trainer function, **trainer config**. Everything explained for the dataset function applied to trainer function as well.

In [9]:
# specify trainer function as string (the function must be importable) as well as the trainer config
trainer_fn = "nnfabrik.training.toy_trainer_fn"
trainer_config = dict(epochs=10) # we specify all the inputs except the ones required by nnfabrik

Trainer().add_entry(trainer_fn=trainer_fn, trainer_config=trainer_config, 
                  trainer_fabrikant="Your Name", trainer_comment="A comment about the trainer!");

In [10]:
Trainer()

trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,trainer_config training configuration object,trainer_fabrikant Name of the contributor that added this entry,trainer_comment short description,trainer_ts UTZ timestamp at time of insertion
nnfabrik.training.toy_trainer_fn,637d24c002384d634263be69ca76f0a6,=BLOB=,Your Name,A comment about the trainer!,2020-06-13 10:01:01


## Seed table

Now we have one final table to fill up before we start training our models with all the combinations in Dataset, Model, and Trainer tables. That table is the **Seed** table.

In [11]:
Seed().insert1({'seed': 2020})

In [12]:
Seed()

seed Random seed that is passed to the model- and dataset-builder
2020


## Trainedmodel table

Once we have bunch of trained models, the downstream analysis might be different for each specific project. for this reason, we keep the TrainedModel trables separate from the tables provided in the library. However, the process of creating your own TrainedModel has becomes very easy with the template(s) provided y nnfabrik.

#### Create your TrainedModel table

In [13]:
from nnfabrik.template import TrainedModelBase

In [14]:
@schema
class TrainedModel(TrainedModelBase):
    table_comment = "Trained models"

## Populate (fill up) the TrainedModel table

In [15]:
TrainedModel.populate(display_progress=True)

100%|██████████| 2/2 [00:01<00:00,  1.01it/s]


In [16]:
TrainedModel()

model_fn name of the model function,model_hash hash of the model configuration,dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,seed Random seed that is passed to the model- and dataset-builder,comment short description,score loss,output trainer object's output,fabrikant_name Name of the contributor that added this entry,trainedmodel_ts UTZ timestamp at time of insertion
nnfabrik.models.toy_model_fn,24922759b843076328c4f3b9df3f88d0,nnfabrik.datasets.toy_dataset_fn,9aee736870714f8b7c3cc084087ce886,nnfabrik.training.toy_trainer_fn,637d24c002384d634263be69ca76f0a6,2020,A comment about the trainer!.A comment about the model!.A comment about the dataset!,0.334282,=BLOB=,None,2020-06-13 10:01:07
nnfabrik.models.toy_model_fn,46b0c26ef504ff0eddf55b15550a4f98,nnfabrik.datasets.toy_dataset_fn,9aee736870714f8b7c3cc084087ce886,nnfabrik.training.toy_trainer_fn,637d24c002384d634263be69ca76f0a6,2020,A comment about the trainer!.A comment about the model!.A comment about the dataset!,0.334282,=BLOB=,None,2020-06-13 10:01:08


---

## Any questions?

If you have any questions feel free to contact any of the contributors.